In [5]:
import cv2
import numpy as np
import os

In [9]:
input_folder = "data_mentah"
output_folder = "data_crop"

In [6]:
# Fungsi bantu untuk mengurutkan titik sudut
def order_points(pts):
    pts = pts.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

In [7]:
# Fungsi transformasi perspektif
def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]
    ], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

In [10]:
# Loop semua file gambar di folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
        path = os.path.join(input_folder, filename)
        image = cv2.imread(path)
        resized = cv2.resize(image, (700, int(image.shape[0] * 700 / image.shape[1])))
        gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edged = cv2.Canny(blurred, 75, 200)

        # Temukan kontur dokumen
        contours, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]

        doc_cnt = None
        for c in contours:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            if len(approx) == 4:
                doc_cnt = approx
                break

        # Jika dokumen ditemukan, transformasi dan simpan
        if doc_cnt is not None:
            warped = four_point_transform(resized, doc_cnt)
            output_path = os.path.join(output_folder, f"scanned_{filename}")
            cv2.imwrite(output_path, warped)
            print(f"[✓] {filename} -> {output_path}")
        else:
            print(f"[!] Tidak ditemukan dokumen: {filename}")

[✓] data_a1.jpg -> data_crop\scanned_data_a1.jpg
[✓] data_a10.jpg -> data_crop\scanned_data_a10.jpg
[✓] data_a100.jpg -> data_crop\scanned_data_a100.jpg
[✓] data_a101.jpg -> data_crop\scanned_data_a101.jpg
[✓] data_a102.jpg -> data_crop\scanned_data_a102.jpg
[✓] data_a103.jpg -> data_crop\scanned_data_a103.jpg
[✓] data_a104.jpg -> data_crop\scanned_data_a104.jpg
[✓] data_a105.jpg -> data_crop\scanned_data_a105.jpg
[✓] data_a107.jpg -> data_crop\scanned_data_a107.jpg
[✓] data_a108.jpg -> data_crop\scanned_data_a108.jpg
[✓] data_a109.jpg -> data_crop\scanned_data_a109.jpg
[✓] data_a11.jpg -> data_crop\scanned_data_a11.jpg
[✓] data_a110.jpg -> data_crop\scanned_data_a110.jpg
[!] Tidak ditemukan dokumen: data_a111.jpg
[✓] data_a112.jpg -> data_crop\scanned_data_a112.jpg
[✓] data_a113.jpg -> data_crop\scanned_data_a113.jpg
[✓] data_a114.jpg -> data_crop\scanned_data_a114.jpg
[✓] data_a115.jpg -> data_crop\scanned_data_a115.jpg
[✓] data_a116.jpg -> data_crop\scanned_data_a116.jpg
[✓] data_a1